# FLEKS Python Visualization Toolkit: IDL Data

flekspy is a Python package for processing FLEKS data. This notebook focuses on handling data in the IDL format.

## FLEKS data format

* Field: *.out format or AMREX built-in format, whose directory name is assumed to end with "_amrex"
* PIC particle: AMREX built-in format 
* Test particle: binary data format

## Importing the package

In [ ]:
import flekspy

## Downloading demo data

If you don't have FLEKS data to start with, you can download demo field data with the following:

In [ ]:
from flekspy.util import download_testfile

url = "https://raw.githubusercontent.com/henry2004y/batsrus_data/master/batsrus_data.tar.gz"
download_testfile(url, "data")

## Loading data

`flekspy.load` is the interface to read files of all formats. It returns a different object for different formats. IDL format data are processed into XArray data structures:

In [ ]:
file = "data/1d__raw_2_t25.60000_n00000258.out"
ds = flekspy.load(file)
ds

The coordinates can be accessed via

In [ ]:
ds.coords

The variables can be accessed via

In [ ]:
ds.var

and individual variables can be accessed through keys or properties:

In [ ]:
ds["p"]
ds.p

### Unstructured data

Unstructured data is handled with [Xugrid](https://deltares.github.io/xugrid/index.html), with support of regridding and partitioning.

In [ ]:
file = "data/bx0_mhd_6_t00000100_n00000352.out"
dsu = flekspy.load(file)
dsu


### Interpolating data

Variable interpolation is supported via [XArray](https://docs.xarray.dev/en/stable/user-guide/interpolation.html). For 1D data, simply provide the coordinate:

In [ ]:
ds["Rho"].interp(x=100)

For multidimensional interpolation, it can be easily extended as

In [ ]:
file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
ds["rhoS0"].interp(x=-28000.0, y =0.0)

If you want to interpolate for all variables at a single location,

In [ ]:
ds.interp(x=-28000.0, y =0.0)

## Visualizing fields

Thanks to XArray's Matplotlib wrapper, all Matplotlib's plotting functionalities are directly supported.

In [ ]:
import matplotlib.pyplot as plt

file = "data/1d__raw_2_t25.60000_n00000258.out"
ds = flekspy.load(file)

fig, axs = plt.subplots(2, 1, constrained_layout=True, sharex=True, sharey=True)
ds.p.plot(ax=axs[0])
ds.Bx.plot(ax=axs[1])
plt.show()

By default, XArray's 2D plotting functions map the first dimension to the vertical y-axis and the second dimension to the horizontal x-axis. To overwrite this default behavior, we can explicitly set the `x` and `y` arguments:

In [ ]:
file = "data/z=0_fluid_region0_0_t00001640_n00010142.out"
ds = flekspy.load(file)
fig, axs = plt.subplots(
    3, 1, figsize=(10, 6), constrained_layout=True, sharex=True, sharey=True
)
ds.Bx.plot.pcolormesh(ax=axs[0], x="x", y="y")
ds.By.plot.pcolormesh(ax=axs[1], x="x", y="y")
ds.Bz.plot.pcolormesh(ax=axs[2], x="x", y="y")
plt.show()

Unstructured IDL format output plotting is supported directly by [Xugrid](https://deltares.github.io/xugrid/examples/plotting.html). Note that here we need an additional level `ugrid` compared to direct plotting via `xarray`.

In [ ]:
dsu.Rho.ugrid.plot.contourf()
plt.show()